# NiN块
NiN块是NiN中的基础块，由一个卷积层+两个充当全连接层的1x1卷积层串联而成。

In [2]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding), # 卷积层 
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), # 1x1卷积层
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), # 1x1卷积层
        nn.ReLU())
    return blk


# 全局池化层
全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现

In [6]:
import torch.nn.functional as F

class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d,self).__init__()
    
    def forward(self,x):
        # x.size() = [1, 10, 5, 5]
        # x.size()[2:] = [5,5]
        return F.avg_pool2d(x,kernel_size = x.size()[2:])

In [7]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2), 
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, stride=1, padding=1),
    # 使用全局平均池化层对每个通道(10个)中所有元素求平均
    GlobalAvgPool2d(), 
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    d2l.FlattenLayer())

构建一个数据样本来查看每一层的输出形状：

In [11]:
X = torch.rand(2, 1, 224, 224) #batch_size = 2 , channels = 1 , width = 224, height = 224
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)
    


0 output shape:  torch.Size([2, 96, 54, 54])
1 output shape:  torch.Size([2, 96, 26, 26])
2 output shape:  torch.Size([2, 256, 26, 26])
3 output shape:  torch.Size([2, 256, 12, 12])
4 output shape:  torch.Size([2, 384, 12, 12])
5 output shape:  torch.Size([2, 384, 5, 5])
6 output shape:  torch.Size([2, 384, 5, 5])
7 output shape:  torch.Size([2, 10, 5, 5])
8 output shape:  torch.Size([2, 10, 1, 1])
9 output shape:  torch.Size([2, 10])
